In [ ]:
# set up notebook for cloud

# on colab, you'll probably need to restart runtime after running the commands below

#! git clone https://github.com/riveSunder/yuca.git
#! pip install -e ./yuca/

# install PYSR and set up julia
#! pip install pysr
#! wget https://julialang-s3.julialang.org/bin/linux/x64/1.6/julia-1.6.6-linux-x86_64.tar.gz
#! tar -xf julia-1.6.6-linux-x86_64.tar.gz


In [ ]:
# steps for setting up pysr to use julia in colab
if(0): 
    import os
    print(os.environ["PATH"])
    os.environ["PATH"] = "/content/julia-1.6.6/bin/:" + os.environ["PATH"]
    import pysr
    pysr.install()

In [ ]:
import numpy as np
import torch

from pysr import PySRRegressor


from yuca.lenia import Lenia
from yuca.zoo.librarian import Librarian
from yuca.utils import seed_all


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 256

import matplotlib.animation
import IPython

In [ ]:
def get_lenia_fn(neighborhood_conv, growth_fn, dt):
    """
    given the convolution and growth functions, and a step size dt
    return a function that implements a continuous CA under Lenia framework
    """
    
    def update_ca_fn(grid):
        
        nbhd = neighborhood_conv(grid)
        growth = growth_fn(nbhd)
        
        #grid = torch.tensor(np.clip(grid.clone().detach() + dt * growth, 0.0, 1.0)).clone().detach().to(grid.dtype).clone().detach()
        grid = torch.clamp(grid + dt*growth, 0, 1.0).double().to(grid.dtype)
        
        return grid
    
    return update_ca_fn

# functions facilitating IPython animation
# initializing the plot
def plot_grid(grid):
    
    global subplot_0 
    
    fig, ax = plt.subplots(1,1, figsize=(6,6), facecolor="white")
    
    subplot_0 = ax.imshow(grid.numpy().squeeze(), cmap="magma", interpolation="nearest")
    
    ax.set_xticklabels("")
    ax.set_yticklabels("")
    
    return fig, ax

# updating the figure during animation
def update_fig(ii):
    
    global subplot_0
    global grid
    global update_ca_fn
    
    grid = update_ca_fn(grid)
    
    subplot_0.set_array(grid.numpy().squeeze())
    



In [ ]:
# user configurable options

use_toy_data = False

# whether to get input x/y dataset from CA simulation
use_ca_dynamics = True

# Gaussian and the Lenia Orbium params
mu = 0.15
sigma = 0.015
gaussian = lambda x: np.exp(-((x -mu) / sigma)**2)
growth_fn = lambda x: 2 * gaussian(x) - 1


# seed pseudorandom number generators
seed_all(42)

In [ ]:
if use_toy_data:
    pass
else:
    
    ca = Lenia()
    ca.restore_config("orbium.npy")
    ca.no_grad()

    x = np.arange(-0.1,1.5, 0.001)
    fig, ax = plt.subplots(1,2, gridspec_kw={"width_ratios": [0.2, 0.8]}, figsize=(12,3))
    ax[0].imshow(ca.neighborhood_kernels.squeeze(), cmap="magma")
    ax[0].set_title("neighborhood kernel")
    ax[1].plot(x, ca.genesis_fns[0](x))
    ax[1].set_title("growth function")
    plt.show()


    #grid[:,:, -64:, :64] = torch.rand(64,64)
    if use_ca_dynamics:
        
        grid = torch.zeros(1,1,128,128)
        my_steps = 32
        num_patterns = 5
        dim_rand = 54
        lib = Librarian()

        grid[:,:, :dim_rand, :] = torch.rand(dim_rand, grid.shape[-1])

        pattern, _ = lib.load("orbium_orbium000")

        for ii in range(num_patterns):
            grid[:,:,25*ii:25*ii+pattern.shape[-2], 25*ii:25*ii + pattern.shape[-1]] = torch.tensor(pattern)

        plt.figure(figsize=(6,6))
        plt.imshow(grid.squeeze(), cmap="magma")

        for step in range(my_steps):
            #neighborhood = ca.neighborhood_conv(grid)
            #old_grid = 1.0 * grid
            grid = ca(grid)

        #grid[:,:, :dim_rand, -dim_rand:] = torch.rand(dim_rand,dim_rand)

        neighborhood = ca.neighborhood_conv(grid)
        old_grid = 1.0 * grid
        grid = ca(grid)

        plt.figure(figsize=(6,6))
        plt.imshow(neighborhood.squeeze(), cmap="magma")

        plt.figure(figsize=(6,6))
        plt.imshow(old_grid.squeeze(), cmap="magma")
        plt.show()

       
    else:
        # use random uniform inputs to ca
        grid = torch.rand(1, 1, 128, 128)
        grid_ramp = torch.tensor(np.arange(0,1,1/grid.shape[-1])).reshape(1,1,1,grid.shape[-1])
        
        grid = (grid * grid_ramp).to(grid.dtype)

        neighborhood = ca.neighborhood_conv(grid)
        old_grid = 1.0 * grid
        grid = ca(grid)

        plt.figure(figsize=(6,6))
        plt.imshow(neighborhood.squeeze(), cmap="magma")

        plt.figure(figsize=(6,6))
        plt.imshow(old_grid.squeeze(), cmap="magma")
        plt.show()
        

    x0 = old_grid.numpy().ravel().reshape(-1,1)
    x1 = neighborhood.numpy().ravel().reshape(-1,1)

    #data_x = np.append(x0, x1, axis=1)
    data_x = x1 
    data_y = ca.genesis_fns[0](data_x) #grid.ravel().reshape(-1,1)


    print(x0.shape, x1.shape, data_x.shape, data_y.shape)


In [ ]:

model = PySRRegressor(
    niterations=10,
    binary_operators=["+", "*"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "inv(x) = 1/x"  # Custom operator (julia syntax)
    ],
    model_selection="best",
    loss="loss(x, y) = (x - y)^2",  # Custom loss function (julia syntax)
)

In [ ]:
idx = np.random.choice(16384, 5000)

model.fit(data_x[idx], data_y[idx])

In [ ]:
print(model)
model.set_params(extra_sympy_mappings={'inv': lambda x: 1 / x})

In [ ]:
# 
sr_fn = lambda x: model.predict(x.ravel()[:,None]).reshape(x.shape)

In [ ]:
model.get_best()["equation"]

In [ ]:
# validate the dynamics. Do patterns behave as expected? 
num_frames = 128
grid = torch.zeros(1,1,96,96)

lib =  Librarian()
ca = Lenia()
ca.restore_config("orbium.npy")
ca.no_grad()

pattern, _ = lib.load("orbium_orbium000")
pattern_start = grid.shape[-2]//4
grid[:,:,pattern_start:pattern_start+pattern.shape[-2],\
         pattern_start:pattern_start+pattern.shape[-1]] = torch.tensor(pattern)

update_ca_fn  = get_lenia_fn(ca.neighborhood_conv, sr_fn, ca.dt)

fig, ax = plot_grid(grid)

plt.close("all")

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=50).to_jshtml())

In [ ]:
# visualize the SR growth function
my_x = data_x[idx]

plt.figure(figsize=(12,8))

plt.plot(my_x, ca.genesis_fns[0](my_x), "o", color="r", label="Orbium growth fn")
plt.plot(my_x, sr_fn(my_x), "x", color="b", alpha=0.1, 
         label="PYSR-learned function")


plt.title(f"pysr best eqn: {model.get_best()['equation']}")
plt.legend()
plt.show()


In [ ]:
# save animation
matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=50).save("pysr_lenia_orbium.gif")

In [ ]:
# standard lenia orbium rules? 
grid = torch.zeros(1,1,96,96)

lib =  Librarian()
ca = Lenia()
ca.restore_config("orbium.npy")
ca.no_grad()

pattern, _ = lib.load("orbium_orbium000")
pattern_start = grid.shape[-2]//4
grid[:,:,pattern_start:pattern_start+pattern.shape[-2],\
         pattern_start:pattern_start+pattern.shape[-1]] = torch.tensor(pattern)


update_ca_fn  = get_lenia_fn(ca.neighborhood_conv, ca.genesis_fns[0], ca.dt)

fig, ax = plot_grid(grid)

plt.close("all")

IPython.display.HTML(matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=50).to_jshtml())

In [ ]:
# save animation
matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=50).save("standard_lenia_orbium.gif")